#Belajar Pyspark - Dataframe View dan SQL

Dalam notebook ini kita akan belajar mengenai cara membuat temporary view dan menjalankan SQL query menggunakan view tersebut.

In [2]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=71685b5eeeab9791b41403938d837a66f34e23fc1f77f59dcececc7b92976e3a
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [4]:
spark = SparkSession.builder.appName("Belajar PySpark - Temp View & SQL").getOrCreate()

Sebelumnya kita buat DataFrame dari sebuah python list, lalu kita akan lakukan transformasi terhadap DataFrame tersebut.


In [5]:
data = [['Agus','F',100],['Windy','F',200],
        ['Budi','B',200],['Dina','F',150],
        ['Bayu','F',50],['Dedi','B',50]]

kolom = ["nama","kode_jurusan","nilai"]

df = spark.createDataFrame(data,kolom)

df.show()

+-----+------------+-----+
| nama|kode_jurusan|nilai|
+-----+------------+-----+
| Agus|           F|  100|
|Windy|           F|  200|
| Budi|           B|  200|
| Dina|           F|  150|
| Bayu|           F|   50|
| Dedi|           B|   50|
+-----+------------+-----+



##Membuat Temporary View

In [10]:
df.createOrReplaceTempView("mahasiswa")

In [8]:
df2 = df.filter(df.kode_jurusan == 'F')
df2.createTempView("mahasiswa")

AnalysisException: ignored

In [9]:
df2.createOrReplaceTempView("mahasiswa")

##Menjalankan Query SQL

In [13]:
df.createOrReplaceTempView("mahasiswa")

In [14]:
spark.sql("select * from mahasiswa").show()

+-----+------------+-----+
| nama|kode_jurusan|nilai|
+-----+------------+-----+
| Agus|           F|  100|
|Windy|           F|  200|
| Budi|           B|  200|
| Dina|           F|  150|
| Bayu|           F|   50|
| Dedi|           B|   50|
+-----+------------+-----+



In [15]:
spark.sql("select * from mahasiswa where kode_jurusan = 'F'").show()

+-----+------------+-----+
| nama|kode_jurusan|nilai|
+-----+------------+-----+
| Agus|           F|  100|
|Windy|           F|  200|
| Dina|           F|  150|
| Bayu|           F|   50|
+-----+------------+-----+



###Agregasi

In [ ]:
spark.sql("""SELECT kode_jurusan,
                    max(nilai) as max_nilai,
                    min(nilai) as min_nilai
             FROM mahasiswa
             GROUP BY kode_jurusan""").show()

+------------+---------+---------+
|kode_jurusan|max_nilai|min_nilai|
+------------+---------+---------+
|           F|      200|       50|
|           B|      200|       50|
+------------+---------+---------+



###Join Reference

In [16]:
ref = [['F','Fisika','MIPA'],['B','Biologi','MIPA'],
       ['A', 'Akuntansi', 'Ekonomi']]
kolom = ["kode_jurusan","nama_jurusan","nama_fakultas"]
df_ref = spark.createDataFrame(ref,kolom)
df_ref.show()

+------------+------------+-------------+
|kode_jurusan|nama_jurusan|nama_fakultas|
+------------+------------+-------------+
|           F|      Fisika|         MIPA|
|           B|     Biologi|         MIPA|
|           A|   Akuntansi|      Ekonomi|
+------------+------------+-------------+



In [17]:
df_ref.createOrReplaceTempView("jurusan")

In [18]:
spark.sql("""SELECT * FROM mahasiswa
          LEFT JOIN jurusan
          ON mahasiswa.kode_jurusan=jurusan.kode_jurusan""").show()

+-----+------------+-----+------------+------------+-------------+
| nama|kode_jurusan|nilai|kode_jurusan|nama_jurusan|nama_fakultas|
+-----+------------+-----+------------+------------+-------------+
| Agus|           F|  100|           F|      Fisika|         MIPA|
|Windy|           F|  200|           F|      Fisika|         MIPA|
| Budi|           B|  200|           B|     Biologi|         MIPA|
| Dina|           F|  150|           F|      Fisika|         MIPA|
| Bayu|           F|   50|           F|      Fisika|         MIPA|
| Dedi|           B|   50|           B|     Biologi|         MIPA|
+-----+------------+-----+------------+------------+-------------+



##Global Temporary View

In [19]:
df.createOrReplaceGlobalTempView("mhs")

In [20]:
spark.sql("SELECT * FROM global_temp.mhs").show()

+-----+------------+-----+
| nama|kode_jurusan|nilai|
+-----+------------+-----+
| Agus|           F|  100|
|Windy|           F|  200|
| Budi|           B|  200|
| Dina|           F|  150|
| Bayu|           F|   50|
| Dedi|           B|   50|
+-----+------------+-----+

